## Imports

In [1]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine


# Options
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
%matplotlib inline

# Convenience for working with external src code files
%load_ext autoreload
%autoreload 2



NOT_APLIC_STR = "NA_SS"
NOT_APLIC_NUM = -999.0
OTHER = "OTHER" 

In [2]:
path = '../data/raw/The-Database-of-Political-Institutions-2020-DPI2020/'
countries = pd.read_stata(path+'DPI2020_stata13.dta')

countries.fillna(np.nan, inplace=True) # Use Numpy's NaN instead of Panda's for consistency
countries['year'] = countries.year.dt.year # no need to use DT format when it is actually just an integer for the year, otherwise, leads to formatting inconsistencies downstream

# determined to be unvaluable based on extensive EDA. Not worthwhile to explain each choice
# refer to data dictionary to infer it is a proper choice
remove_these_features = countries.iloc[:, 25:76].columns
countries.drop(remove_these_features, axis=1, inplace=True)

## Understanding columns

- ifs - "IFS" country code
- system - Presidential (0), Assembly-elected president (1), parliamentry (2)
- yrsoffc - how many years has chief executive been in office?
- finittrm - is there a finite term? (0, 1)
- yrcurnt - years left in current term
- multpl - if there are formal restraints on an executive’s term (NA if not), can s/he serve additional term(s) following the current one?
- military - is chief exeuctive a military officer?
- defmin - is defense minister a military officer?
- PERCENT1 - president got what % of votes in the 1st/only round?
- PERCENTL - president got what % of votes in the final round? (na if no runoff)
- PRTYIN - party of chief executive has been how long in office
- EXECME - name of party, if any **remove** (too many distinct values)
- EXECRLC - executive is Right (1), Left (3), Center (2), No information (0), No executive (NA)
- EXECNAT - Nationalist (0, 1)
- EXECRURL - "Rural" issues listed as key component of party's platform? **remove** nearly no 1s
- EXECREG - "Regional" issues listed as key component of party's platform? **remove** nearly no 1s
- EXECREL - Executive religion
- EXECAGE - time since party formation (under same name)
- ALLHOUSE - does party of executive control all relevant houses?
- NONCHIEF - party affiliation of the one not called "Chief Executive" (in systems w/both non-ceremonial PM and president) **remove**
- TOTALSEATS - total seats in legislature. Includes gov1seat, gov2seat, gov3seat, opp1seat, opp2seat, opp3seat, govothst, oppthst, numul)
- GOV1ME, GOV1SEAT GOV1[ETC] - Descriptors of largest party. Too granular to bother with in this analysis since there are already aggregate features that account for the same data. **remove**
- OPPMAJH - does one opposition party have an absolute majority in House?
- OPPMAJS - does one opposition party have an absolute majority in Senate?
- DATELEG - month when parliamentary elections were held **remove**
- DATEEXEC - month when presidential elections were held **remove**
- LEGELEX - legislative election this year?
- EXELEC - executive election this year?
- LIEC, EIEC - Legislative, Executive Index of Electoral Completiveness (see data dictionary, valuable metric)
- MDMH, MDMS - Mean District Magnitude House, Senate
- PLURALTY - government is plurality?
- PR - proportional representation?
- HOUSESYS, SENSYS - House, Senate electorical rule
- THRESH - vote threshold for representation *in proportional representation system*
- DHONDT - is the D'Hondt system used? **remove**
- CL - are closed lists used? 
- GQ, GQI - gender quota, whether it was implemented
- SELECT - method for selecting election candidates **remove** (almost no values)

--------

- AUTON - are there autonomous regions? (federalism)
- MUNI - are municipal governments locally elected?
- STATE - are there state/providence governments locally elected? (important!)
- AUTHOR - do the state/provinves have authority over taxing, spending, or legislating? (important!)
- STCONST - are the constituencies of the senators the states/provides? **remove**
- GWNO - no idea, not included in dictioanry **remove**
- NUMGOV - total number of seats held by all government parties **remove**
- NUMVOTE - vote share of ruling government party  
- NUMOPP - vote share of opposition government party **remove**
- FRAUD - were vote fraud or candidate intimidation serious enough to affect the outcome of elections
- MAJ - Margin of Majority (important!)
- PARTYAGE - average age of parties
- HERFGOV, HERFOPP - Herfindahl index government (sum of squared seat shares of all parties in the government) [probably a valuable metric]
- HERFTOT - same as above, for all, but very few values. **remove**

--------

- TENLONG - longest tenure of a veto player **remove in favor of strict**
- TENLONG_STRICT - uses TENLONG, restricted to fewer certain leaders 
- TENSHORT - shortest tensure of a veto player **remove in favor of strict**
- TENSHORT_strict - uses TENSHORT, restricted to fewer leaders
- CHECKS - checks and balances
- CHECKS_LAX - **remove**, corresponds to same group as TENLONG (removed), as opposed to CHECKS, which maps to people in TENLONG_STRICT
- STABS, STABS_STRICT - "Stability". use STRICT, as explained in TENLONG.
- STABNS, STABNS_STRICT - similar to STABS, **remove**
- POLARIZ, POLARIZ_STRICT - maximum polarization between the executive party and the four principle parties of the legislature. Use STRING similar to above. 

In [3]:
# Replace "placeholder" NaN values (as defined by data dictionary) 
# with one consistent np.nan across dataset for consistency

countries.replace(-999, NOT_APLIC_NUM, inplace=True)
countries.replace(-999.0, NOT_APLIC_NUM, inplace=True)
countries.replace('-999', NOT_APLIC_NUM, inplace=True)
countries.replace('-999.0', NOT_APLIC_NUM, inplace=True)
countries.replace('NA', NOT_APLIC_STR, inplace=True)



# One-off cases based on close reading of data dictionary
countries.system.replace(-999, NOT_APLIC_STR, inplace=True)
countries.system = countries.system.astype('object')
countries.execrlc.replace(0.0, NOT_APLIC_STR, inplace=True)
countries.execrlc = countries.execrlc.astype('object')
countries.execrel.replace(0.0, OTHER, inplace=True)
countries.execrel.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.execrel = countries.execrel.astype('object')
countries.percent1.replace(999.0, NOT_APLIC_NUM, inplace=True)
countries.percentl.replace(-99.0, NOT_APLIC_NUM, inplace=True)
countries.execrlc.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.oppmajs.replace(999.0, NOT_APLIC_NUM, inplace=True)
countries.legelec.replace(12.0, NOT_APLIC_NUM, inplace=True)
countries.liec.replace(0.0, NOT_APLIC_NUM, inplace=True)
countries.eiec.replace(0.0, NOT_APLIC_NUM, inplace=True)
countries.housesys.replace('PR', 'Proportional', inplace=True)
countries.housesys.replace(0.5, NOT_APLIC_STR, inplace=True)
countries.housesys.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.housesys = countries.housesys.astype('object')
countries.sensys.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.sensys.replace(-888.0, NOT_APLIC_STR, inplace=True)
countries.sensys.replace(0.5, NOT_APLIC_STR, inplace=True)
countries.sensys.replace('PR', 'Proportional', inplace=True)
countries.sensys = countries.sensys.astype('object')
countries.thresh.replace(-9999.0, NOT_APLIC_NUM, inplace=True)
countries.select.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.fraud.replace(0.0, 'OppositionBanned', inplace=True)
countries.fraud.replace(1.0, 'OppositionSuppressed', inplace=True)
countries.fraud.replace(-999.0, NOT_APLIC_STR, inplace=True)
countries.fraud.fillna(NOT_APLIC_STR, inplace=True)
countries.muni.replace('Legislature and executive locally elected', 1.0, inplace=True)
countries.muni.replace('Legislature locally elected', 0.5, inplace=True)
countries.muni.replace('No local elections', 0.0, inplace=True)
countries.muni = countries.muni.astype('float')

countries.state.replace('Legislature and executive locally elected', 1.0, inplace=True)
countries.state.replace('Legislature locally elected', 0.5, inplace=True)
countries.state.replace('No local elections', 0.0, inplace=True)
countries.state = countries.state.astype('float')

countries.maj.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.maj.fillna(NOT_APLIC_NUM, inplace=True)
countries.maj = countries.maj.astype('float')
countries.partyage.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.partyage.fillna(NOT_APLIC_NUM, inplace=True)
countries.partyage = countries.partyage.astype('float')
countries.herfgov.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.herfgov = countries.herfgov.astype('float')
countries.herfopp.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.herfopp = countries.herfopp.astype('float')



countries.frac = countries.frac.astype('object')
countries.frac.fillna(NOT_APLIC_NUM, inplace=True)
countries.frac.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)

countries.frac = countries.frac.astype('object')
countries.frac.fillna(NOT_APLIC_NUM, inplace=True)
countries.frac.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)

countries.oppfrac = countries.oppfrac.astype('object')
countries.oppfrac.fillna(NOT_APLIC_NUM, inplace=True)
countries.oppfrac.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)

countries.govfrac = countries.govfrac.astype('object')
countries.govfrac.fillna(NOT_APLIC_NUM, inplace=True)
countries.govfrac.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)

countries.tensys_strict.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.tensys_strict = countries.tensys_strict.astype('float')

countries.checks.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.checks = countries.checks.astype('float')

countries.stabs_strict.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.stabs_strict = countries.stabs_strict.astype('float')

countries.tenlong_strict.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.tenlong_strict = countries.tenlong_strict.astype('float')

countries.tenshort_strict.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.tenshort_strict = countries.tenshort_strict.astype('float')

countries.tenshort_strict.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)
countries.tenshort_strict = countries.tenshort_strict.astype('float')

countries.polariz.replace(NOT_APLIC_STR, NOT_APLIC_NUM, inplace=True)



In [4]:
# countries[['countryname', 'ifs']].drop_duplicates()

### Country names codes

Make consistent with other data sources to enable merging. Minor differences in spelling, abbreviations, casing, etc. make this necessary. Process completed in simple Excel file for simplicity. Import that lookup table to convert to new naming schemes.

In [5]:
countries.loc[(countries.ifs=='CZE')]# | (countries.ifs=='CSK')]

,countryname,ifs,year,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,military,defmin,percent1,percentl,prtyin,execme,execrlc,execnat,execrurl,execreg,execrel,execage,allhouse,nonchief,totalseats,oppmajh,oppmajs,dateleg,dateexec,legelec,exelec,liec,eiec,mdmh,mdms,ssh,pluralty,pr,housesys,sensys,thresh,dhondt,cl,select,gq,gqi,fraud,auton,muni,state,author,stconst,gwno,numgov,numvote,numopp,oppvote,maj,partyage,herfgov,herfopp,herftot,frac,oppfrac,govfrac,tensys_strict,tensys,checks_lax,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
1902,Czech Rep.,CZE,1991,Assembly-Elected President,1.0,1.0,4.0,1.0,1.0,1.0,0.0,1.0,-999.0,-999.0,1.0,Civic F.,NA_SS,0.0,0.0,0.0,OTHER,2.0,0.0,-999.0,150,1.0,1.0,-999.0,-999.0,0.0,0.0,7.0,7.0,NaN,75.0,0.25,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,1.0,315.0,127.0,63.500000,23.0,13.700000,0.846667,24.333334,0.502852,1.000000,NaN,0.620157,0.000000,0.501094,1.0,1.0,4.0,4.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0
1903,Czech Rep.,CZE,1992,Assembly-Elected President,2.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,2.0,Civic F.,NA_SS,0.0,0.0,0.0,OTHER,3.0,0.0,-999.0,150,1.0,1.0,6.0,-999.0,1.0,0.0,7.0,7.0,NaN,75.0,0.25,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,0.0,315.0,127.0,63.500000,23.0,13.700000,0.846667,25.333334,0.502852,1.000000,NaN,0.620157,0.000000,0.501094,2.0,2.0,4.0,4.0,0.000000,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0
1904,Czech Rep.,CZE,1993,Parliamentary,1.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,1.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,2.0,0.0,independent,200,1.0,1.0,-999.0,-999.0,0.0,0.0,7.0,7.0,25.00,75.0,0.29,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,1.0,315.0,105.0,41.939999,95.0,38.950001,0.525000,3.000000,0.562086,0.235900,NaN,0.795829,0.772228,0.442125,3.0,3.0,6.0,6.0,0.500000,0.5,0.666667,0.666667,1.0,1.0,1.0,1.0,2.0
1905,Czech Rep.,CZE,1994,Parliamentary,2.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,2.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,3.0,0.0,independent,200,1.0,1.0,-999.0,-999.0,0.0,0.0,7.0,7.0,25.00,75.0,0.29,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,1.0,315.0,105.0,41.939999,95.0,38.950001,0.525000,4.000000,0.562086,0.235900,NaN,0.795829,0.772228,0.442125,4.0,4.0,6.0,6.0,0.000000,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
1906,Czech Rep.,CZE,1995,Parliamentary,3.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,3.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,4.0,0.0,independent,200,1.0,1.0,-999.0,-999.0,0.0,0.0,7.0,7.0,25.00,1.0,0.29,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,0.0,315.0,105.0,41.939999,95.0,38.950001,0.525000,5.000000,0.562086,0.235900,NaN,0.795829,0.772228,0.442125,5.0,5.0,6.0,6.0,0.000000,0.0,0.0,0.0,3.0,3.0,3.0,3.0,2.0
1907,Czech Rep.,CZE,1996,Parliamentary,4.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,4.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,5.0,0.0,independent,200,NaN,NaN,5.0,-999.0,1.0,0.0,7.0,7.0,25.00,1.0,0.29,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,1.0,315.0,105.0,41.939999,95.0,38.950001,0.525000,6.000000,0.562086,0.235900,NaN,0.795829,0.772228,0.442125,6.0,6.0,6.0,6.0,0.000000,0.0,0.0,0.0,4.0,4.0,4.0,4.0,2.0
1908,Czech Rep.,CZE,1997,Parliamentary,5.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,1.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,6.0,0.0,independent,200,NaN,NaN,-999.0,-999.0,0.0,0.0,7.0,7.0,25.00,1.0,0.29,1.0,1.0,Proportional,Plurality,5.0,0.0,0.0,NaN,0.0,0.0,OppositionBanned,0.0,NaN,0.5,1.0,0.0,315.0,99.0,44.060001,101.0,44.779999,0.495000,7.000000,0.522090,0.443976,NaN,0.762663,0.561584,0.482787,7.0,7.0,7.0,7.0,0.200000,0.2,0.25,0.25,5.0,5.0,1.0,1.0,2.0
1909,Czech Rep.,CZE,1998,Parliamentary,6.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,-999.0,-999.0,6.0,ODS/KDS,Right,0.0,0.0,0.0,Christian,7.0,0.0,independent,200,1.0,0.0,6.0,-999.0,1.0,0.0,7.0,7.0,25.00,1.0,0.29,1

In [6]:
code_lookup = pd.read_excel('../src/country_codes_governments.xlsx')
name_dict  = dict(zip(code_lookup.combo_original, code_lookup.country_final))
scode_dict = dict(zip(code_lookup.combo_original, code_lookup.scode_final))

countries['combo'] = countries.countryname + '_' + countries.ifs
countries['countryname_final'] = countries.combo.map(name_dict)
countries['scode_final'] = countries.combo.map(scode_dict)

display(countries[['combo', 'countryname', 'countryname_final', 'ifs', 'scode_final']].drop_duplicates())
countries.drop(['countryname', 'ifs'], axis=1, inplace=True)

,combo,countryname,countryname_final,ifs,scode_final
0,Turk Cyprus_0,Turk Cyprus,IGNORE,0,INGORE
46,Afghanistan_AFG,Afghanistan,Afghanistan,AFG,AFG
92,Angola_AGO,Angola,Angola,AGO,ANG
138,Albania_ALB,Albania,Albania,ALB,ALB
184,UAE_ARE,UAE,United Arab Emirates,ARE,UAE
230,Argentina_ARG,Argentina,Argentina,ARG,ARG
276,Armenia_ARM,Armenia,Armenia,ARM,ARM
322,Australia_AUS,Australia,Australia,AUS,AUS
368,Austria_AUT,Austria,Austria,AUT,AUT
414,Azerbaijan_AZE,Azerbaijan,Azerbaijan,AZE,AZE


In [7]:
# Normalize percentages to [0, 1] instead of [0, 100]
def normalize_to_percent(zero_to_100):
    if_not_999 = lambda x: x/100 if x != -999.0 else x
    percentage = zero_to_100.astype('float').apply(if_not_999)
    return percentage



countries.polariz = countries.polariz.astype('float')
countries.percent1 = normalize_to_percent(countries.percent1)
countries.percentl = normalize_to_percent(countries.percentl)
countries.numvote = normalize_to_percent(countries.numvote)
countries.oppvote = normalize_to_percent(countries.oppvote)

In [8]:
# Based on manual review of the data dictionary, explained briefly above
remove_these_features = ['nonchief', 'dateleg', 'dateexec', 'dhondt', 'select', 'stconst',
                        'gwno', 'numgov', 'numopp', 'herftot', 'tenlong', 'tenshort',
                        'checks_lax', 'stabns', 'stabns_strict', 'execme', 'tensys', 'stabs', 'execrurl',
                        'execreg']
countries.drop(remove_these_features, axis=1, inplace=True)

## Feature Engineering

In [9]:
# Combine PERCENT1 and PERCENTL since they cover similar content since 
# PERCENTL is more meaningful in cases where it exists, but it often doesn't exist

last_else_first = lambda first, last: last if last != NOT_APLIC_NUM else first

countries['percent1'].fillna(NOT_APLIC_NUM, inplace=True)
countries['percentl'].fillna(NOT_APLIC_NUM, inplace=True)

countries['percent'] = list(map(last_else_first, countries['percent1'], countries['percentl']))
countries['percent'].replace(NOT_APLIC_NUM, np.nan, inplace=True)
countries.drop(['percent1', 'percentl'], axis=1, inplace=True)

# Column-by-column cleaning complete
## Now, create unique identifier for subsequent merge with other datasets

In [10]:
countries['year_scode'] = countries.year.astype('str') + '_' + countries.scode_final
countries.drop(['year', 'combo'], axis=1, inplace=True)
countries

,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,military,defmin,prtyin,execrlc,execnat,execrel,execage,allhouse,totalseats,oppmajh,oppmajs,legelec,exelec,liec,eiec,mdmh,mdms,ssh,pluralty,pr,housesys,sensys,thresh,cl,gq,gqi,fraud,auton,muni,state,author,numvote,oppvote,maj,partyage,herfgov,herfopp,frac,oppfrac,govfrac,tensys_strict,checks,stabs_strict,tenlong_strict,tenshort_strict,polariz,countryname_final,scode_final,percent,year_scode
0,NA_SS,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,NaN,-999.0,NA_SS,-999.0,NA_SS,-999.0,-999.0,0,-999.0,-999.0,0.0,0.0,-999.0,-999.0,NaN,-999.0,-999.00,-999.0,-999.0,NA_SS,NA_SS,2.0,-999.0,0.0,0.0,OppositionBanned,0.0,0.0,0.0,0.0,0.0,0.0,-999.000000,-999.0,-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,-999.0,NaN,-999.0,-999.0,-999.0,IGNORE,INGORE,NaN,1975_INGORE
1,Presidential,1.0,1.0,0.0,1.0,1.0,1.0,NaN,NaN,-999.0,NA_SS,1.0,OTHER,NaN,1.0,0,0.0,-999.0,0.0,1.0,1.0,2.0,NaN,-999.0,-999.00,-999.0,-999.0,NA_SS,NA_SS,2.0,-999.0,0.0,0.0,NA_SS,NaN,NaN,NaN,0.0,0.0,0.0,-999.000000,-999.0,-999.0,-999.000000,-999.000000,-999.000000,-999.0,1.0,1.0,-999.00,1.0,1.0,0.0,IGNORE,INGORE,NaN,1976_INGORE
2,Presidential,2.0,1.0,4.0,1.0,1.0,1.0,NaN,NaN,-999.0,Right,1.0,OTHER,NaN,1.0,40,0.0,-999.0,0.0,0.0,7.0,6.5,NaN,NaN,NaN,NaN,NaN,NA_SS,NA_SS,2.0,NaN,0.0,0.0,OppositionBanned,0.0,0.0,0.0,0.0,0.0,0.0,0.725000,-999.0,1.0,0.371901,0.457692,0.690909,0.0,1.0,3.0,0.00,2.0,1.0,0.0,IGNORE,INGORE,NaN,1977_INGORE
3,Presidential,3.0,1.0,3.0,1.0,1.0,1.0,NaN,NaN,-999.0,Right,1.0,OTHER,NaN,1.0,40,0.0,-999.0,0.0,0.0,7.0,6.5,NaN,NaN,NaN,NaN,NaN,NA_SS,NA_SS,2.0,NaN,0.0,0.0,OppositionBanned,0.0,0.0,0.0,0.0,0.0,0.0,0.725000,-999.0,1.0,0.371901,0.457692,0.690909,0.0,2.0,3.0,0.00,3.0,2.0,0.0,IGNORE,INGORE,NaN,1978_INGORE
4,Presidential,4.0,1.0,2.0,1.0,1.0,1.0,NaN,NaN,-999.0,Right,1.0,OTHER,NaN,1.0,40,0.0,-999.0,0.0,0.0,7.0,6.5,NaN,NaN,NaN,NaN,NaN,NA_SS,NA_SS,2.0,NaN,0.0,0.0,OppositionBanned,0.0,0.0,0.0,0.0,0.0,0.0,0.725000,-999.0,1.0,0.371901,0.457692,0.690909,0.0,3.0,3.0,0.00,4.0,3.0,0.0,IGNORE,INGORE,NaN,1979_INGORE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195,Presidential,29.0,1.0,2.0,3.0,1.0,1.0,0.0,0.0,36.0,NA_SS,1.0,OTHER,53.0,1.0,210,0.0,0.0,0.0,0.0,6.0,7.0,1.0,1.0,0.35,1.0,0.0,Plurality,Plurality,-999.0,-999.0,1.0,0.0,OppositionBanned,0.0,NaN,NaN,NaN,0.0,0.0,0.761905,35.0,1.0,1.000000,0.366781,0.000000,0.0,36.0,4.0,0.00,29.0,3.0,0.0,Zimbabwe,ZIM,0.6188,2016_ZIM
8196,Presidential,30.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0,37.0,NA_SS,1.0,OTHER,54.0,1.0,210,0.0,0.0,0.0,0.0,6.0,7.0,1.0,1.0,0.35,1.0,0.0,Plurality,Plurality,-999.0,-999.0,1.0,0.0,OppositionBanned,0.0,NaN,0.0,NaN,0.0,0.0,0.761905,36.0,1.0,1.000000,0.366781,0.000000,0.0,37.0,4.0,0.00,30.0,4.0,0.0,Zimbabwe,ZIM,0.6188,2017_ZIM
8197,Presidential,1.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,38.0,NA_SS,1.0,OTHER,55.0,1.0,210,0.0,0.0,1.0,1.0,6.0,7.0,1.0,1.0,0.35,1.0,0.0,Plurality,Plurality,-999.0,-999.0,1.0,0.0,OppositionSuppressed,0.0,NaN,0.0,NaN,0.0,0.0,0.761905,37.0,1.0,1.000000,0.366781,0.000000,0.0,38.0,4.0,0.25,5.0,1.0,0.0,Zimbabwe,ZIM,NaN,2018_ZIM
8198,Presidential,2.0,1.0,4.0,3.0,1.0,1.0,0.0,0.0,39.0,NA_SS,1.0,OTHER,56.0,1.0,210,0.0,0.0,0.0,0.0,7.0,7.0,1.0,1.0,0.35,1.0,0.0,Plurality,Plurality,-999.0,-999.0,1.0,0.0,OppositionSuppressed,0.0,NaN,NaN,NaN,0.0,0.0,0.690476,28.5,1.0,0.969238,0.435270,0.031250,0.0,39.0,4.0,0.00,6.0,2.0,0.0,Zimbabwe,ZIM,0.5143,2019_ZIM


## Create identifier to map countries in "Protests" to "Countries"

In [11]:
# # Import "Protests" dataset
# engine = create_engine('sqlite:///../data/processed/protests.db')
# with engine.begin() as connection:
#     protests = pd.read_sql('SELECT * FROM protests', connection)

In [12]:
# # Identify cases where a protest_country isn't in the countries_country
# match = {}
# no_match = {}
# for country in protests.country.unique():
#     if country not in countries.countryname.unique():
#         no_match[country] = None
# no_match

##### Given the above list, manually create a dictionary to find the "Countries" country name corresponding to the "Protest" country name, since it is likely a difference in syntax/spelling/etc

In [13]:
# print(countries.countryname.unique())

In [14]:
# no_match['Dominican Republic'] = 'Dom. Rep.'
# no_match['United Kingdom'] = 'UK'
# no_match['Germany'] = 'FRG/Germany'
# no_match['Germany West'] = 'FRG/Germany'
# no_match['Germany East'] = 'GDR'
# no_match['Czechoslovakia'] = 'Czech Rep.'
# no_match['Czech Republic'] = 'Czech Rep.'
# no_match['Slovak Republic'] = 'Slovakia'
# no_match['Kosovo'] = None # No corresponding country in "Countries" dataset
# no_match['Serbia'] = None # No corresponding country in "Countries" dataset
# no_match['Bosnia'] = 'Bosnia-Herz' # No corresponding country in "Countries" dataset
# no_match['Serbia and Montenegro'] = None # No corresponding country in "Countries" dataset
# no_match['Montenegro'] = None # No corresponding country in "Countries" dataset
# no_match['USSR'] = 'Soviet Union'
# no_match['Cape Verde'] = 'C. Verde Is.'
# no_match['Equatorial Guinea'] = 'Eq. Guinea'
# no_match['Ivory Coast'] = "Cote d'Ivoire"
# no_match['Central African Republic'] = 'Cent. Af. Rep.'
# no_match['Congo Brazzaville'] = 'Congo'
# no_match['Congo Kinshasa'] = 'Congo (DRC)'
# no_match['South Africa'] = 'S. Africa'
# no_match['Comoros'] = 'Comoro Is.'
# no_match['United Arab Emirate'] = 'UAE'
# no_match['China'] = 'PRC'
# no_match['North Korea'] = 'PRK'
# no_match['South Korea'] = 'ROK'
# no_match['Timor Leste'] = 'Timor-Leste'
# no_match['Papua New Guinea'] = 'P. N. Guinea'


# keys = no_match.keys()
# for country in list(no_match):
#     if no_match[country] == None:
#         del no_match[country]
        
# no_match

In [15]:
# country_idb = []
# for country in protests.country:
#     if country in countries.countryname.unique():
#         country_idb.append(country)
#     elif country in no_match.keys():
#         country_idb.append(no_match[country])
#     else:
#         country_idb.append(np.nan)
        
# protests['country_idb'] = country_idb

In [16]:
# # Count the number of missing values
# # These rows will be dropped since they don't have corresponding data in IDB for countryname
# print('Rows w/missing data:', protests.country_idb.isna().sum())

In [17]:
# protests['id_idb'] = protests.startyear.astype('str')+' '+protests.country_idb
# protests.dropna(inplace=True)
# protests.shape

In [18]:
# merged = pd.merge(protests, countries, how='left', on='id_idb')

# cols_to_drop = ['id_idb', 'country_idb', 'year']
# merged.drop(cols_to_drop, axis=1, inplace=True)
# merged

# Export data to SQL 

In [19]:
# FILL IN MISSING VALUES W/CORRECT DATA TYPE
fill_w_object_type = ['system', 'execrlc', 'execrel', 'housesys', 'sensys', 'fraud']

fill_w_num_type = ['yrsoffc', 'finittrm', 'yrcurnt', 'termlimit', 'reelect',
       'multpl', 'military', 'defmin', 'prtyin', 'execnat', 'execage', 'allhouse', 'totalseats', 'oppmajh', 'oppmajs',
       'legelec', 'exelec', 'liec', 'eiec', 'mdmh', 'mdms', 'ssh', 'pluralty',
       'pr', 'thresh', 'cl', 'gq', 'gqi',
       'auton', 'muni', 'state', 'author', 'numvote', 'oppvote', 'maj',
       'partyage', 'herfgov', 'herfopp', 'frac', 'oppfrac', 'govfrac',
       'tensys_strict', 'checks', 'stabs_strict', 'tenlong_strict',
       'tenshort_strict', 'polariz', 'percent','year_scode']

for column in fill_w_object_type:
    countries[column].fillna(NOT_APLIC_STR, inplace=True)
for column in fill_w_num_type:
    countries[column].fillna(NOT_APLIC_NUM, inplace=True)


# USE COMMON CATEGORIES OF ALL-ENCOMPASSING FEATURES TO CATEGORIZE MISSING VALUES
no_govt_data = countries.loc[(countries.system == NOT_APLIC_STR) & 
                             (countries.yrcurnt == NOT_APLIC_NUM)]# & 
                             #(countries.fraud == NOT_APLIC_STR)]
no_govt_data_ind = list(no_govt_data.index)
countries['no_govt_data'] = [int(x in no_govt_data_ind) for x in countries.index]

# Drop rows without data
drop_missing = countries.loc[countries['no_govt_data']==1].index
countries.drop(drop_missing, inplace=True)
countries.drop('no_govt_data', axis=1, inplace=True)

In [20]:
countries.rename(columns={'countryname_final': 'country_govt', 
                'scode_final': 'scode_govt'}, inplace=True)

engine = create_engine('sqlite:///../data/processed/governments.db')

with engine.begin() as connection:
    countries.to_sql(name='governments', con=connection, if_exists='replace', index=False)

In [21]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7669 entries, 1 to 8199
Data columns (total 57 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   system           7669 non-null   object 
 1   yrsoffc          7669 non-null   float64
 2   finittrm         7669 non-null   float64
 3   yrcurnt          7669 non-null   float64
 4   termlimit        7669 non-null   float64
 5   reelect          7669 non-null   float64
 6   multpl           7669 non-null   float64
 7   military         7669 non-null   float64
 8   defmin           7669 non-null   float64
 9   prtyin           7669 non-null   float64
 10  execrlc          7669 non-null   object 
 11  execnat          7669 non-null   float64
 12  execrel          7669 non-null   object 
 13  execage          7669 non-null   float64
 14  allhouse         7669 non-null   float64
 15  totalseats       7669 non-null   int16  
 16  oppmajh          7669 non-null   float64
 17  oppmajs       